## DuckDB and dataframe-jdbc

[DuckDB](https://duckdb.org/), for now, is an officially unsupported JDBC source for DataFrame.
However, a source being officially unsupported does not mean that it is not possible to use it with DataFrame :).

For DuckDB, we actually have two ways to support it:

The first one is to use `dataframe-arrow`, as DuckDB has a [bridge to Arrow](https://duckdb.org/docs/stable/clients/java.html#arrow-methods). [Click here for en example of how to do it](https://github.com/Kotlin/dataframe/blob/62b48942b1aef35f939f2d0aff407872028fd177/dataframe-arrow/src/test/kotlin/org/jetbrains/kotlinx/dataframe/io/ArrowKtTest.kt#L637). This might work for you, but it is less flexible, as you're forced to use Arrow types.

The second way, which we will show here, is to use `dataframe-jdbc`, as DuckDB has a [JDBC driver](https://duckdb.org/docs/stable/clients/java.html) too.
This is a more direct approach and shows how you can add other JDBC sources as well, so without any further ado, let's get started!

We're adding the DuckDB dependency before adding DataFrame to make sure classloading works well

In [1]:
USE {
    dependencies {
        implementation(group = "org.duckdb", artifact = "duckdb_jdbc", version = "1.2.2.0")
    }
}

And to make sure we use the latest version of DataFrame, we'll use the `%useLatestDescriptors` magic.

In [2]:
%useLatestDescriptors
%use dataframe

DuckDB can connect to a specific database by path or, if no path is supplied, run in memory!
This is what we'll use for this example.

[Read more about connecting to DuckDB](https://duckdb.org/docs/stable/clients/java.html)

In [3]:
val URL = "jdbc:duckdb:"

Let's insert some example data into our database.
Usually, a database already has data in it, or, in DuckDB's case, it's connected via an [extension](https://duckdb.org/docs/stable/core_extensions/overview) to another existing database.

In [4]:
import java.sql.DriverManager

val connection = DriverManager.getConnection(URL)

connection.prepareStatement(
    """
CREATE TABLE IF NOT EXISTS test_table (
    id INTEGER PRIMARY KEY,
    name VARCHAR,
    age INTEGER,
    salary DOUBLE,
    hire_date DATE,
    pets VARCHAR[],
    map MAP(INT, VARCHAR),
)
    """.trimIndent(),
).executeUpdate()

connection.prepareStatement(
    """
    INSERT INTO test_table (id, name, age, salary, hire_date, pets, map)
    VALUES
        (1, 'John Doe', 30, 50000.00, '2020-01-15', ['Pookie'], MAP { 1: 'value1', 200: 'value2' }),
        (2, 'Jane Smith', 28, 55000.00, '2021-03-20', [], MAP {}),
        (3, 'Bob Johnson', 35, 65000.00, '2019-11-10', ['Rex', 'Garfield'], MAP { 3: 'value3', 4: 'value4' }),
        (4, 'Alice Brown', 32, 60000.00, '2020-07-01', ['Mia'], MAP { 5: 'value5', 6: 'value6' }),
        (5, 'Charlie Smith', 25, 70000.00, '2022-02-01', ['Max'], MAP { 7: 'value7', 8: 'value8' }),
        (6, 'David Smith', 27, 80000.00, '2022-03-01', ['Max', 'Charlie'], MAP { 9: 'value9', 10: 'value10' }),
        (7, 'Eve Smith', 24, 90000.00, '2022-04-01', ['Max', 'Charlie', 'David'], MAP { 11: 'value11', 12: NULL }),
    """.trimIndent(),
).executeUpdate()


7

### Attempt 1

Following the [custom JDBC database documentation](https://kotlin.github.io/dataframe/readsqlfromcustomdatabase.html), it seems we need to create our own `DbType` object.

We will skip the `convertSqlTypeToKType` and `convertSqlTypeToColumnSchemaValue` functions for now. Let's see how the defaults fare.

In [5]:
import org.jetbrains.kotlinx.dataframe.io.db.DbType
import org.jetbrains.kotlinx.dataframe.io.getSchemaForAllSqlTables
import org.jetbrains.kotlinx.dataframe.io.readAllSqlTables
import org.jetbrains.kotlinx.dataframe.schema.ColumnSchema
import java.sql.ResultSet
import kotlin.reflect.KType

object DuckDb1 : DbType("duckdb") {

    /** the name of the class of the DuckDB JDBC driver */
    override val driverClassName = "org.duckdb.DuckDBDriver"

    /**
     * How a column type from JDBC, [tableColumnMetadata], is read in Java/Kotlin.
     * The returned type must exactly follow [ResultSet.getObject] of your specific database's JDBC driver.
     * Returning `null` defer the implementation to the default one (which may not always be correct).
     *
     * TODO We'll do this later
     */
    override fun convertSqlTypeToKType(tableColumnMetadata: TableColumnMetadata): KType? = null

    /**
     * How a column from JDBC should be represented as DataFrame (value) column
     * See [convertSqlTypeToKType].
     *
     * TODO We'll do this later
     */
    override fun convertSqlTypeToColumnSchemaValue(tableColumnMetadata: TableColumnMetadata): ColumnSchema? = null

    /**
     * How to filter out system tables from user-created ones when using
     * [DataFrame.readAllSqlTables] and [DataFrame.getSchemaForAllSqlTables].
     *
     * The names of these can sometimes be found in the specific JDBC integration.
     */
    override fun isSystemTable(tableMetadata: TableMetadata): Boolean =
        tableMetadata.schemaName?.lowercase()?.contains("information_schema") == true ||
            tableMetadata.schemaName?.lowercase()?.contains("system") == true ||
            tableMetadata.name.lowercase().contains("system_")

    /**
     * How to retrieve the correct table metadata when using
     * [DataFrame.readAllSqlTables] and [DataFrame.getSchemaForAllSqlTables].
     * The names of these can be found in the [DatabaseMetaData] implementation of the DuckDB JDBC integration.
     */
    override fun buildTableMetadata(tables: ResultSet): TableMetadata =
        TableMetadata(
            name = tables.getString("TABLE_NAME"),
            schemaName = tables.getString("TABLE_SCHEM"),
            catalogue = tables.getString("TABLE_CAT"),
        )
}

Now, with the same connection, let's see if we can read the data from our database into DataFrame!

Since our `DbType` object is a custom one, we need to explicitly set it as the `dbType` parameter.

In [6]:
val df1 = DataFrame.readSqlTable(connection, tableName = "test_table", dbType = DuckDb1)
df1

The problem is found in one of the loaded libraries: check library converters (fields callbacks)
Error compiling code:
@DataSchema
interface _DataFrameType {
    val age: Int
    val hire_date: java.util.Date
    val id: Int
    val map: Any
    val name: String
    val pets: Array<T>
    val salary: Double
}

val ColumnsScope<_DataFrameType>.age: DataColumn<Int> @JvmName("_DataFrameType_age") get() = this["age"] as DataColumn<Int>
val DataRow<_DataFrameType>.age: Int @JvmName("_DataFrameType_age") get() = this["age"] as Int
val ColumnsScope<_DataFrameType>.hire_date: DataColumn<java.util.Date> @JvmName("_DataFrameType_hire_date") get() = this["hire_date"] as DataColumn<java.util.Date>
val DataRow<_DataFrameType>.hire_date: java.util.Date @JvmName("_DataFrameType_hire_date") get() = this["hire_date"] as java.util.Date
val ColumnsScope<_DataFrameType>.id: DataColumn<Int> @JvmName("_DataFrameType_id") get() = this["id"] as DataColumn<Int>
val DataRow<_DataFrameType>.id: Int @JvmName("_Da

id,name,age,salary,hire_date,pets,map
1,John Doe,30,50000.000000,2020-01-15,[Pookie],"{1=value1, 200=value2}"
2,Jane Smith,28,55000.000000,2021-03-20,[ ],{}
3,Bob Johnson,35,65000.000000,2019-11-10,"[Rex, Garfield]","{3=value3, 4=value4}"
4,Alice Brown,32,60000.000000,2020-07-01,[Mia],"{5=value5, 6=value6}"
5,Charlie Smith,25,70000.000000,2022-02-01,[Max],"{7=value7, 8=value8}"
6,David Smith,27,80000.000000,2022-03-01,"[Max, Charlie]","{9=value9, 10=value10}"
7,Eve Smith,24,90000.000000,2022-04-01,"[Max, Charlie, David]","{11=value11, 12=null}"


Aside from some errors, that seems to work! Let's check which types are correct. There's usually a difference between the types returned by the JDBC driver and the types provided by DataFrame, causing runtime exceptions due to the mismatch.

We can check this by explicitly inferring the types of a dataframe and seeing if they match the types of the original:

In [7]:
import kotlin.reflect.full.isSubtypeOf

df1.schema().print()
println()
df1.inferType().schema().print()
println()

(df1.columnTypes() zip  df1.inferType().columnTypes()).mapNotNull { (provided, inferred) ->
    if (inferred.isSubtypeOf(provided)) null else "$inferred is not a subtype of $provided"
}

id: Int
name: String
age: Int
salary: Double
hire_date: java.util.Date
pets: Array<Any>
map: Any

id: Int
name: String
age: Int
salary: Double
hire_date: java.time.LocalDate
pets: Array<Any>
map: java.util.HashMap<*, *>



[java.time.LocalDate is not a subtype of java.util.Date]

Yup, it looks like we have some more differences! Let's see if we can fix them in our custom `DbType` object.

### Attempt 2

We will look at the implementation of `ResultSet.getObject` in DuckDB's JDBC driver, and use that to fix the types that mismatch the DataFrame JDBC implementation.

In [8]:
import org.duckdb.DuckDBResultSetMetaData
import org.duckdb.DuckDBColumnType.*
import org.duckdb.JsonNode
import org.jetbrains.kotlinx.dataframe.io.db.DbType
import org.jetbrains.kotlinx.dataframe.io.getSchemaForAllSqlTables
import org.jetbrains.kotlinx.dataframe.io.readAllSqlTables
import org.jetbrains.kotlinx.dataframe.schema.ColumnSchema
import java.math.BigInteger
import java.sql.ResultSet
import java.time.LocalTime
import java.util.UUID
import kotlin.reflect.*
import kotlin.reflect.full.withNullability

object DuckDb2 : DbType("duckdb") {

    /** the name of the class of the DuckDB JDBC driver */
    override val driverClassName = "org.duckdb.DuckDBDriver"

    /**
     * How a column type from JDBC, [tableColumnMetadata], is read in Java/Kotlin.
     * The returned type must exactly follow [ResultSet.getObject] of your specific database's JDBC driver.
     * Returning `null` defer the implementation to the default one (which may not always be correct).
     *
     * Following [org.duckdb.DuckDBVector.getObject].
     */
    override fun convertSqlTypeToKType(tableColumnMetadata: TableColumnMetadata): KType? =
        when (DuckDBResultSetMetaData.TypeNameToType(tableColumnMetadata.sqlTypeName)) {
            TINYINT -> typeOf<Byte>()
            SMALLINT -> typeOf<Short>()
            HUGEINT -> typeOf<BigInteger>()
            UHUGEINT -> typeOf<BigInteger>()
            UTINYINT -> typeOf<Short>()
            USMALLINT -> typeOf<Int>()
            UINTEGER -> typeOf<Long>()
            UBIGINT -> typeOf<BigInteger>()
            TIME -> typeOf<java.time.LocalTime>()
            DATE -> typeOf<java.time.LocalDate>()
            JSON -> typeOf<JsonNode>()
            BLOB -> typeOf<java.sql.Blob>()
            UUID -> typeOf<UUID>()
            MAP -> typeOf<Map<Any, Any?>>() // TODO we'll add more specific types here later
            LIST, ARRAY -> typeOf<java.sql.Array>()
            STRUCT -> typeOf<java.sql.Struct>()
            UNION -> typeOf<Any>()
            VARCHAR -> typeOf<String>()
            UNKNOWN, BIT, INTERVAL, ENUM -> typeOf<String>()
            else -> null // use DataFrame JDBC defaults
        }?.withNullability(tableColumnMetadata.isNullable)


    /**
     * How a column from JDBC should be represented as DataFrame (value) column
     * See [convertSqlTypeToKType].
     */
    override fun convertSqlTypeToColumnSchemaValue(tableColumnMetadata: TableColumnMetadata): ColumnSchema? =
        convertSqlTypeToKType(tableColumnMetadata)?.let {
            // The current DataFrame JDBC implementation will always create value columns as no type-conversion is done
            ColumnSchema.Value(it)
        }

    /**
     * How to filter out system tables from user-created ones when using
     * [DataFrame.readAllSqlTables] and [DataFrame.getSchemaForAllSqlTables].
     *
     * The names of these can sometimes be found in the specific JDBC integration.
     */
    override fun isSystemTable(tableMetadata: TableMetadata): Boolean =
        tableMetadata.schemaName?.lowercase()?.contains("information_schema") == true ||
            tableMetadata.schemaName?.lowercase()?.contains("system") == true ||
            tableMetadata.name.lowercase().contains("system_")

    /**
     * How to retrieve the correct table metadata when using
     * [DataFrame.readAllSqlTables] and [DataFrame.getSchemaForAllSqlTables].
     * The names of these can be found in the [DatabaseMetaData] implementation of the DuckDB JDBC integration.
     */
    override fun buildTableMetadata(tables: ResultSet): TableMetadata =
        TableMetadata(
            name = tables.getString("TABLE_NAME"),
            schemaName = tables.getString("TABLE_SCHEM"),
            catalogue = tables.getString("TABLE_CAT"),
        )
}

In [9]:
val df2 = DataFrame.readSqlTable(connection, tableName = "test_table", dbType = DuckDb2)
df2

id,name,age,salary,hire_date,pets,map
1,John Doe,30,50000.000000,2020-01-15,[Pookie],"{1=value1, 200=value2}"
2,Jane Smith,28,55000.000000,2021-03-20,[],{}
3,Bob Johnson,35,65000.000000,2019-11-10,"[Rex, Garfield]","{3=value3, 4=value4}"
4,Alice Brown,32,60000.000000,2020-07-01,[Mia],"{5=value5, 6=value6}"
5,Charlie Smith,25,70000.000000,2022-02-01,[Max],"{7=value7, 8=value8}"
6,David Smith,27,80000.000000,2022-03-01,"[Max, Charlie]","{9=value9, 10=value10}"
7,Eve Smith,24,90000.000000,2022-04-01,"[Max, Charlie, David]","{11=value11, 12=null}"


In [10]:
df2.schema().print()
println()
df2.inferType().schema().print()
println()

(df2.columnTypes() zip df2.inferType().columnTypes()).mapNotNull { (provided, inferred) ->
    if (inferred.isSubtypeOf(provided)) null else "$inferred is not a subtype of $provided"
}

id: Int
name: String
age: Int
salary: Double
hire_date: java.time.LocalDate
pets: java.sql.Array
map: Map<Any, Any?>

id: Int
name: String
age: Int
salary: Double
hire_date: java.time.LocalDate
pets: org.duckdb.DuckDBArray
map: java.util.HashMap<Any, Any?>



[]

That seems to work better!

### Attempt 3

There's just one last thing we can improve, and that's nested types.
This is completely optional, as DataFrame has `.inferType()`. That relies on runtime inference, however,
which costs performance. Providing types as specific as possible will let us avoid that.

As we cannot do conversions upon reading from JDBC [yet](https://github.com/Kotlin/dataframe/issues/1266),
we cannot do anything about the `Struct`, or `JsonNode` types.
Similarly, we cannot do anything about the `Array` and `List` types [yet](https://github.com/Kotlin/dataframe/issues/1273).

However, we can provide nested `Map` types!

The DuckDB type can be found in `TableColumnMetadata.sqlTypeName`.

DuckDB [MAP types](https://duckdb.org/docs/stable/sql/data_types/map) are represented as a `MAP(X, Y)` type, where `X` and `Y` are the types of the keys and values in the map. They can be nested arbitrarily deep, meaning you can get a: `MAP(MAP(MAP(INT, VARCHAR[][]), STRUCT(i INT, v VARCHAR)), MAP(INT, VARCHAR))` and we should still be able to parse it!

This means we're going to need a special parsing function that splits the `MAP` type into its key and value types at the right comma.

In [11]:
import org.duckdb.DuckDBResultSetMetaData
import org.duckdb.DuckDBColumnType.*
import org.duckdb.JsonNode
import org.jetbrains.kotlinx.dataframe.io.db.DbType
import org.jetbrains.kotlinx.dataframe.io.getSchemaForAllSqlTables
import org.jetbrains.kotlinx.dataframe.io.readAllSqlTables
import org.jetbrains.kotlinx.dataframe.schema.ColumnSchema
import java.math.BigDecimal
import java.math.BigInteger
import java.sql.ResultSet
import java.time.LocalTime
import java.time.OffsetDateTime
import java.time.OffsetTime
import java.util.UUID
import kotlin.reflect.*
import kotlin.reflect.full.createType
import kotlin.reflect.full.withNullability

object DuckDb : DbType("duckdb") {

    /** the name of the class of the DuckDB JDBC driver */
    override val driverClassName = "org.duckdb.DuckDBDriver"

    /**
     * How a column type from JDBC, [tableColumnMetadata], is read in Java/Kotlin.
     * The returned type must exactly follow [ResultSet.getObject] of your specific database's JDBC driver.
     * Returning `null` defer the implementation to the default one (which may not always be correct).
     *
     * Following [org.duckdb.DuckDBVector.getObject].
     */
    override fun convertSqlTypeToKType(tableColumnMetadata: TableColumnMetadata): KType =
        tableColumnMetadata.sqlTypeName.toKType(tableColumnMetadata.isNullable)


    /**
     * How a column from JDBC should be represented as DataFrame (value) column
     * See [convertSqlTypeToKType].
     */
    override fun convertSqlTypeToColumnSchemaValue(tableColumnMetadata: TableColumnMetadata): ColumnSchema =
        convertSqlTypeToKType(tableColumnMetadata).let {
            // The current DataFrame JDBC implementation will always create value columns as no type-conversion is done
            ColumnSchema.Value(it)
        }

    /**
     * Follows exactly [org.duckdb.DuckDBVector.getObject].
     *
     * I added a "// dataframe-jdbc" comment for all types that are covered correctly by
     * [org.jetbrains.kotlinx.dataframe.io.makeCommonSqlToKTypeMapping] at the moment, however, to cover
     * all nested types, we'll use a full type-map for all [DuckDB types][DuckDBColumnType] exactly.
     */
    @Suppress("ktlint:standard:blank-line-between-when-conditions")
    private fun String.toKType(isNullable: Boolean): KType {
        val sqlTypeName = this
        return when (DuckDBResultSetMetaData.TypeNameToType(sqlTypeName)!!) {
            BOOLEAN -> typeOf<Boolean>() // dataframe-jdbc
            TINYINT -> typeOf<Byte>()
            SMALLINT -> typeOf<Short>()
            INTEGER -> typeOf<Int>() // dataframe-jdbc
            BIGINT -> typeOf<Long>() // dataframe-jdbc
            HUGEINT -> typeOf<BigInteger>()
            UHUGEINT -> typeOf<BigInteger>()
            UTINYINT -> typeOf<Short>()
            USMALLINT -> typeOf<Int>()
            UINTEGER -> typeOf<Long>()
            UBIGINT -> typeOf<BigInteger>()
            FLOAT -> typeOf<Float>() // dataframe-jdbc
            DOUBLE -> typeOf<Double>() // dataframe-jdbc
            DECIMAL -> typeOf<BigDecimal>() // dataframe-jdbc
            TIME -> typeOf<java.time.LocalTime>()
            TIME_WITH_TIME_ZONE -> typeOf<OffsetTime>() // dataframe-jdbc
            DATE -> typeOf<java.time.LocalDate>()
            TIMESTAMP, TIMESTAMP_MS, TIMESTAMP_NS, TIMESTAMP_S -> typeOf<java.sql.Timestamp>() // dataframe-jdbc
            TIMESTAMP_WITH_TIME_ZONE -> typeOf<OffsetDateTime>() // dataframe-jdbc
            JSON -> typeOf<JsonNode>()
            BLOB -> typeOf<java.sql.Blob>()
            UUID -> typeOf<UUID>()
            MAP -> {
                val (key, value) = parseMapTypes(sqlTypeName)
                Map::class.createType(
                    listOf(
                        KTypeProjection.invariant(key.toKType(isNullable = false)), // keys cannot be nullable in DuckDB
                        KTypeProjection.covariant(value.toKType(isNullable = true)), // values can be nullable in DuckDB
                    ),
                )
            }
            LIST, ARRAY -> typeOf<java.sql.Array>() // TODO requires #1266 and #1273 for specific types
            STRUCT -> typeOf<java.sql.Struct>() // TODO requires #1266 for specific types
            UNION -> typeOf<Any>() // Cannot handle this in Kotlin
            VARCHAR -> typeOf<String>()
            UNKNOWN, BIT, INTERVAL, ENUM -> typeOf<String>()
            // as a bonus, we have an exhaustive when now :)
        }.withNullability(isNullable)
    }

    /** Splits "MAP(X, Y)" into "X" and "Y", taking nested parentheses into account. */
    fun parseMapTypes(typeString: String): Pair<String, String> {
        if (!typeString.startsWith("MAP(") || !typeString.endsWith(")")) {
            error("invalid MAP type: $typeString")
        }
        val content = typeString.removeSurrounding("MAP(", ")")

        // Find the comma that separates key and value types
        var parenCount = 0
        var commaIndex = -1
        for (i in content.indices) {
            when (content[i]) {
                '(' -> parenCount++
                ')' -> parenCount--
                ',' -> if (parenCount == 0) {
                    commaIndex = i
                    break
                }
            }
        }
        if (commaIndex == -1) error("invalid MAP type: $typeString")
        val keyType = content.take(commaIndex).trim()
        val valueType = content.substring(commaIndex + 1).trim()
        return Pair(keyType, valueType)
    }

    /**
     * How to filter out system tables from user-created ones when using
     * [DataFrame.readAllSqlTables] and [DataFrame.getSchemaForAllSqlTables].
     *
     * The names of these can sometimes be found in the specific JDBC integration.
     */
    override fun isSystemTable(tableMetadata: TableMetadata): Boolean =
        tableMetadata.schemaName?.lowercase()?.contains("information_schema") == true ||
            tableMetadata.schemaName?.lowercase()?.contains("system") == true ||
            tableMetadata.name.lowercase().contains("system_")

    /**
     * How to retrieve the correct table metadata when using
     * [DataFrame.readAllSqlTables] and [DataFrame.getSchemaForAllSqlTables].
     * The names of these can be found in the [DatabaseMetaData] implementation of the DuckDB JDBC integration.
     */
    override fun buildTableMetadata(tables: ResultSet): TableMetadata =
        TableMetadata(
            name = tables.getString("TABLE_NAME"),
            schemaName = tables.getString("TABLE_SCHEM"),
            catalogue = tables.getString("TABLE_CAT"),
        )
}

In [12]:
val df3 = DataFrame.readSqlTable(connection, tableName = "test_table", dbType = DuckDb)
df3

id,name,age,salary,hire_date,pets,map
1,John Doe,30,50000.000000,2020-01-15,[Pookie],"{1=value1, 200=value2}"
2,Jane Smith,28,55000.000000,2021-03-20,[],{}
3,Bob Johnson,35,65000.000000,2019-11-10,"[Rex, Garfield]","{3=value3, 4=value4}"
4,Alice Brown,32,60000.000000,2020-07-01,[Mia],"{5=value5, 6=value6}"
5,Charlie Smith,25,70000.000000,2022-02-01,[Max],"{7=value7, 8=value8}"
6,David Smith,27,80000.000000,2022-03-01,"[Max, Charlie]","{9=value9, 10=value10}"
7,Eve Smith,24,90000.000000,2022-04-01,"[Max, Charlie, David]","{11=value11, 12=null}"


Now we have a correctly typed dataframe with `map: Map<Int, String?>`!

In [13]:
df3.schema().print()
println()
df3.inferType().schema().print()
println()

(df3.columnTypes() zip df3.inferType().columnTypes()).mapNotNull { (provided, inferred) ->
    if (inferred.isSubtypeOf(provided)) null else "$inferred is not a subtype of $provided"
}

id: Int
name: String
age: Int
salary: Double
hire_date: java.time.LocalDate
pets: java.sql.Array
map: Map<Int, String?>

id: Int
name: String
age: Int
salary: Double
hire_date: java.time.LocalDate
pets: org.duckdb.DuckDBArray
map: java.util.HashMap<Int, String?>



[]

With that, we just created a complete `DbType` object for DuckDB in DataFrame!
If we provide this in each call to `DataFrame.readSql*`, we can read our database into a DataFrame.

### Final conversions

Some optional but recommended conversions are:
- Turning the `java.sql.Array`'s into `List<T?>`'s.
- Turning the `java.sql.Struct`'s into `Map<String, T?>`s or a `DataRow<*>`.
- Turning the `Map<String, T?>`'s into a `DataRow<*>`.
- Turning the `JsonNode` into a `String` and `parse()` it or use `DataRow.readJsonStr()`.
- Turning `java.time` instances to `kotlinx.datetime` instances
- Turning `java.sql.Timestamp` into `kotlinx.datetime.LocalDateTime`

Note that most of these rely on runtime inference, which can be costly for large dataframes.

In [20]:
import kotlinx.datetime.toKotlinInstant
import kotlinx.datetime.toKotlinLocalDateTime
import org.duckdb.DuckDBStruct

fun AnyFrame.convertDuckDbTypes() = this
    .convert { colsOf<java.sql.Array>() }.with(infer = Infer.Type) { (it.array as Array<Any?>).toList() }
    .convert { colsOf<java.sql.Struct>() }.with { (it as DuckDBStruct).map }
    .convert { colsOf<Map<String, *>>() }.with {
        // getting a DataRow from Map to create a ColumnGroup with the keys as column names.
        // TODO simplify with #1098 when it's done
        it.mapValues { listOf(it.value) }.toDataFrame().single()
    }
    .convert { colsOf<JsonNode>() }.with { DataRow.readJsonStr(it.toString()) }
    .convert { colsOf<java.time.LocalTime>() }.toLocalTime()
    .convert { colsOf<java.time.LocalDate>() }.toLocalDate()
    .convert { colsOf<java.time.OffsetDateTime>() }.with { it.toInstant().toKotlinInstant() }
    .convert { colsOf<java.sql.Timestamp>() }.with { it.toLocalDateTime().toKotlinLocalDateTime() }

val df = df3.convertDuckDbTypes()

df.schema().print()
df

id: Int
name: String
age: Int
salary: Double
hire_date: kotlinx.datetime.LocalDate
pets: List<String>
map: Map<Int, String?>


id,name,age,salary,hire_date,pets,map
1,John Doe,30,50000.000000,2020-01-15,[Pookie],"{1=value1, 200=value2}"
2,Jane Smith,28,55000.000000,2021-03-20,[ ],{}
3,Bob Johnson,35,65000.000000,2019-11-10,"[Rex, Garfield]","{3=value3, 4=value4}"
4,Alice Brown,32,60000.000000,2020-07-01,[Mia],"{5=value5, 6=value6}"
5,Charlie Smith,25,70000.000000,2022-02-01,[Max],"{7=value7, 8=value8}"
6,David Smith,27,80000.000000,2022-03-01,"[Max, Charlie]","{9=value9, 10=value10}"
7,Eve Smith,24,90000.000000,2022-04-01,"[Max, Charlie, David]","{11=value11, 12=null}"
